In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_pickle("train.pkl")

In [4]:
df = df.sort_values("order_date")
#df

In [5]:
drop_cols = ["order_date", "user_id", "supplier_id", "order_line_item_id", "order_parent_id"]


df = df.drop(drop_cols, axis=1)

In [6]:
train_0 = df[df.is_returned == 0]
train_1 = df[df.is_returned == 1]

In [7]:
train_0.shape

(9402321, 37)

In [8]:
train_1.shape

(1370340, 37)

In [9]:
9402321/1370340

6.86130522352117

In [11]:
inc = train_1.shape[0]
dfs = []

for i in range(7):
    start_idx = i*inc
    end_idx = start_idx + inc
    #print(start_idx, end_idx)
    dfs.append(pd.concat([train_0.iloc[start_idx:end_idx, :], train_1]))

In [20]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, plot_importance
from sklearn.metrics import f1_score, confusion_matrix, classification_report

models = []
for data in dfs:
    X_train, X_val, y_train, y_val = train_test_split(data.drop("is_returned", axis=1),
                                                       data["is_returned"],
                                                       stratify=data["is_returned"],
                                                       random_state=0,
                                                       test_size=0.1)

    model = LGBMClassifier(random_state=0)
    model.fit(X_train, y_train)
    models.append(model)
    preds = model.predict(X_val)
    print(f1_score(y_val, preds))

0.9318378668771722
0.9143039762925946
0.9002131579898118
0.9007006436102819
0.8996903957815975
0.9126097539772233
0.9313159582531632


In [138]:
test = pd.read_pickle("test.pkl")
test.head()

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,...,rate_prod_max,rate_max,rate_prod_mean,rate_mean,returnRate,defective_returnRate,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
88,2021-08-01 20:25:51.462000+00:00,10505160,0,4442,1109528418,723041833,73655340,126823590,99.9375,99.9375,...,5.0,5.0,4.654639,4.654639,0.049683,0.00076,170200.0,43.0,2.0,0.046509
93,2021-08-04 12:26:54.966000+00:00,64880957,0,4442,1114797805,726087440,73655340,126823590,99.9375,74.9375,...,5.0,5.0,4.654639,4.654639,0.049683,0.00076,170200.0,43.0,2.0,0.046509
138,2021-08-03 23:04:19.178000+00:00,908797,0,4442,1114230175,725753274,73655340,126823590,99.9375,74.9375,...,5.0,5.0,4.654639,4.654639,0.049683,0.00076,170200.0,43.0,2.0,0.046509
159,2021-08-06 00:12:47.500000+00:00,4527825,0,4442,1117826558,727894275,73655340,126823590,99.9375,74.9375,...,5.0,5.0,4.654639,4.654639,0.049683,0.00076,170200.0,43.0,2.0,0.046509
177,2021-08-07 22:25:41.777000+00:00,12926531,0,4442,1121135072,729879757,73655340,126823590,99.9375,74.9375,...,5.0,5.0,4.654639,4.654639,0.049683,0.00076,170200.0,43.0,2.0,0.046509


In [139]:
X_test = test[X_train.columns]

In [140]:
for i, clf in enumerate(models):
    preds = clf.predict(X_test)
    test["preds_"+str(i)] = preds

## Voting

In [119]:
#test["preds_final"] = test.iloc[:, -7:].apply(lambda x: x.mode(), axis=1)

In [120]:
#test["preds_final"] = test.iloc[:, -7:].apply(lambda x: x.max(), axis=1)

In [141]:
test["preds_final"] = test.iloc[:, -7:].apply(lambda x: int(x.sum() > 6), axis=1)

In [134]:
test["id"] = test.apply(lambda x: str(x.user_id)+"|"+str(x.product_content_id)+"|"+str(x.order_parent_id), axis=1)

In [144]:
test["preds_final"].value_counts()

0.0    421016
Name: preds_final, dtype: int64

In [135]:
#test = test.groupby("id")["preds_final"].max().reset_index()
#test

,id,preds_final
0,10000099|124684006|726798736,0
1,10000099|92327604|727936988,0
2,10000099|92696330|726906909,0
3,10000099|94935954|726798736,0
4,10000109|93878976|722737629,0
...,...,...
405277,9999967|35527270|727780795,0
405278,9999967|37635011|727780795,0
405279,9999967|79616260|727780795,0
405280,9999967|86570446|727780795,0


In [124]:
test_data = pd.read_csv("test_data.csv")
test_data

,id,expected
0,18115|87073974|726348863,NaN
1,27588|118753298|723169934,NaN
2,29165|35699988|724122443,NaN
3,29969|5893187|722342150,NaN
4,35107|93897418|723421057,NaN
...,...,...
7466,71083899|100019570|728638514,NaN
7467,71106534|35624085|725372928,NaN
7468,71115350|94438742|725291348,NaN
7469,71115350|100017676|725291348,NaN


In [125]:
test_final = test_data.merge(test, on="id", how="left")
test_final

,id,expected,preds_final
0,18115|87073974|726348863,NaN,1
1,27588|118753298|723169934,NaN,1
2,29165|35699988|724122443,NaN,0
3,29969|5893187|722342150,NaN,1
4,35107|93897418|723421057,NaN,0
...,...,...,...
7466,71083899|100019570|728638514,NaN,1
7467,71106534|35624085|725372928,NaN,0
7468,71115350|94438742|725291348,NaN,0
7469,71115350|100017676|725291348,NaN,0


In [126]:
test_final = test_final.drop("expected", axis=1)
test_final.columns = ["id", "expected"]
test_final

,id,expected
0,18115|87073974|726348863,1
1,27588|118753298|723169934,1
2,29165|35699988|724122443,0
3,29969|5893187|722342150,1
4,35107|93897418|723421057,0
...,...,...
7466,71083899|100019570|728638514,1
7467,71106534|35624085|725372928,0
7468,71115350|94438742|725291348,0
7469,71115350|100017676|725291348,0


In [111]:
test_final["expected"] = test_final["expected"].astype(int)

In [127]:
test_final["expected"].value_counts(normalize=True)

0    0.841253
1    0.158747
Name: expected, dtype: float64

In [128]:
test_final["expected"].value_counts()

0    6285
1    1186
Name: expected, dtype: int64

In [129]:
test_final.to_csv("2022_04_19_deneme6.csv", index=False)